In [1]:
from numpy import *
from scipy.sparse import *
from sklearn.preprocessing import *

In [2]:
PlTrackSim = load_npz("target_pl_x_target_tracks_similarity.npz")

In [3]:
PlwithArtist = load_npz("targetPlaylistArtistReduced.npz")

In [4]:
tTracksWithArtist = load_npz("targetTracksArtistReduced.npz")

In [5]:
PlwithArtist = normalize(PlwithArtist, norm="max")

In [6]:
PlTrackSimByArtist = PlwithArtist * tTracksWithArtist.T

In [7]:
artistsTags = load_npz("artist_with_tag_bincount_reduced.npz")
artistsTags = normalize(artistsTags)

In [8]:
P=load_npz("playlist_reduced.npz")
P=normalize(P)

In [9]:
PlArtistSim = P * artistsTags.T

In [10]:
PlTrackSimByArtistTags = PlArtistSim * tTracksWithArtist.T

In [ ]:
finalPlSimH = PlTrackSimByArtist + PlTrackSim 

In [13]:
finalPlSim = finalPlSimH + PlTrackSimByArtistTags

In [14]:
finalPlSim

<10000x32195 sparse matrix of type '<class 'numpy.float64'>'
	with 257059946 stored elements in Compressed Sparse Row format>

In [11]:
target_playlists=genfromtxt("target_playlists.csv",skip_header=1)
playlists_with_tracks=load("playlists_with_tracks.npy")
target_tracks_ordered = load("targetTracksOrdered.npy")
def getsimil(pls, similrow, n):
    maxi = flip(argsort(similrow), axis=0)
    r = []
    for m in maxi:
        if(not isin(target_tracks_ordered[m], pls[1:])):
           r.append(target_tracks_ordered[m])
           if(len(r)==n):
               return r

In [16]:
fname = "tag_artist2_average_similarity.csv"
open(fname,"w").close()
print("playlist_id,track_ids",file=open(fname,"a"))
for pl, simil in zip(playlists_with_tracks, finalPlSim):
    s = str(int(pl[0]))
    s += ","
    r = getsimil(pl, ravel(simil.todense()), 5)
    for el in r:
        s+=str(el)
        s+=" "
    print(s,file=open(fname,"a"))

In [17]:
k = asarray([[3, 0, 1, 0], [1, 1, 2, 0], [0, 0, 1, 3]])

In [22]:
k =normalize(k)

In [23]:
z = asarray([[1, 0, 1, 0], [0, 1, 1, 0], [0, 1, 0, 1]])

In [24]:
dot(k, z.T)

array([[ 1.26491106,  0.31622777,  0.        ],
       [ 1.22474487,  1.22474487,  0.40824829],
       [ 0.31622777,  0.31622777,  0.9486833 ]])

In [36]:
fname = "tag_artist2_intersect_similarity.csv"
open(fname,"w").close()
print("playlist_id,track_ids",file=open(fname,"a"))
iii=0
for pl, similPT, similPTA, similsum in zip(playlists_with_tracks, PlTrackSim, PlArtistSim, finalPlSimH):
    s = str(int(pl[0]))
    s += ","
    r1 = getsimil(pl, ravel(similPT.todense()), 15)
    r2 = getsimil(pl, ravel(similPTA.todense()), 15)
    
    rr = intersect1d(r1, r2, assume_unique=True)
    if(len(rr)!=0):
        iii+=1
    if(len(rr)>=5):
        r = rr.take(range(5))
    else:
        r4 = getsimil(pl, ravel(similsum.todense()), 5)
        r4 = array([el for el in r4 if el not in rr])
        r = concatenate((rr, r4.take(range(5-len(rr)))))
    for el in r:
        s+=str(el)
        s+=" "
    print(s,file=open(fname,"a"))

In [12]:
#modified version with mono artist
monoArtist=load("target_playlist_mono_artist.npy")
monoArtistPlID=monoArtist[:,0]
monoArtistArtistID=monoArtist[:,1]
artistIndexes=load("uniqueArtists_NeededToIndexThe_ArtistReducedMatrices.npy")
artistTracks=load_npz("artists_with_tracksID_ordered_by_occurrencies.npz")

In [21]:
#modified version with mono artist
fname = "MonoArtistCorrect_TagSimilarity.csv"
with open(fname,"a") as myfile:
    myfile.write("playlist_id,track_ids\n")
    for pl, similPT, similsum in zip(playlists_with_tracks, PlTrackSim, finalPlSimH):
        plID=pl[0]
        s = str(int(plID))
        s += ","
        if(isin(plID,monoArtistPlID)):#guarda se c'è un artista solo
            artist=monoArtistArtistID[where(monoArtistPlID==plID)[0][0]]#l'unico artista della playlist
            artistIndex=where(artistIndexes==artist)[0][0]#cerca la sua posizione nella matrice delle tracce
            thisArtistTracks=artistTracks.getrow(artistIndex).data#prende le sue tracce
            rr=array([t for t in thisArtistTracks if t not in pl[1:] and isin(t, target_tracks_ordered)])#prende tutte quelle non già presenti
            
            if(len(rr)>=5):
                r = array(rr).take(range(5))
            else:
                r4 = getsimil(pl, ravel(similsum.todense()), 5)
                r4 = array([el for el in r4 if el not in rr])
                r = concatenate((rr, r4.take(range(5-len(rr)))))
        else:
            r = getsimil(pl, ravel(similsum.todense()), 5)#non c'è un artista solo quindi guarda solo i tag
        for el in r:
            s+=str(el)
            s+=" "
        myfile.write(s + "\n")

In [37]:
k = [1, 2, 3]

In [38]:
e = [1, 0, 4]

In [39]:
iii

78

In [23]:
k = array([el for el in k if el not in e])

In [24]:
k

array([2, 3])

In [13]:
PLNTtags = load_npz("non_target_pl_tags_reduced.npz")
PLNTtags = normalize(PLNTtags)
PLTxPLNT = P * PLNTtags.T

In [14]:
tTracksTags = load_npz("reducedTargetT.npz")

In [ ]:
PLNTxTtracks = PLNTtags * tTracksTags.T

In [ ]:
F = PLTxPLNT * PLNTxTtracks

In [ ]:
Final = F + PlTrackSimByArtist

In [ ]:
#modified version with mono artist
fname = "New Mono_TagSimilarity.csv"
with open(fname,"a") as myfile:
    myfile.write("playlist_id,track_ids\n")
    for pl, similPT, similsum in zip(playlists_with_tracks, F, Final):
        plID=pl[0]
        s = str(int(plID))
        s += ","
        if(isin(plID,monoArtistPlID)):#guarda se c'è un artista solo
            artist=monoArtistArtistID[where(monoArtistPlID==plID)[0][0]]#l'unico artista della playlist
            artistIndex=where(artistIndexes==artist)[0][0]#cerca la sua posizione nella matrice delle tracce
            thisArtistTracks=artistTracks.getrow(artistIndex).data#prende le sue tracce
            rr=array([t for t in thisArtistTracks if t not in pl[1:] and isin(t, target_tracks_ordered)])#prende tutte quelle non già presenti
            
            if(len(rr)>=5):
                r = array(rr).take(range(5))
            else:
                r4 = getsimil(pl, ravel(similsum.todense()), 5)
                r4 = array([el for el in r4 if el not in rr])
                r = concatenate((rr, r4.take(range(5-len(rr)))))
        else:
            r = getsimil(pl, ravel(similsum.todense()), 5)#non c'è un artista solo quindi guarda solo i tag
        for el in r:
            s+=str(el)
            s+=" "
        myfile.write(s + "\n")